기본세팅할 것:
1. 구글 드라이브에 train/test폴더 압축해서 각각 zip파일로 올리기(코랩에서 압축풀어야 세션에 업로드되서 속도가 빠름)
2. 구글 드라이브에 train.csv , test.csv 올리기
3. 자신이 사용하는 경로에 q2l_labeller 폴더 올리기(모듈 import를 위함)

램 25기가로 설정해야 돌아감

해야 할 것들(수정)
1. randaugment로 데이터 증강을 하는 것이 문제가 될 수 있다고 판단.-> 0.6% 좋아졌네.. / 일단 그대로 두고 나중에 여러가지로 제출해보기
2. 배경처리를 개선해야 함. 현재는 그냥 사진 7개 중 랜덤으로 배경을 선택하게 하였으나, 배경도 회전,대칭 등을 통해 다양하게 주어야 함 ->pixaby 에서 저작권없는 무료이미지 8개 + 직접찍은사진7개, 총 15개를 변형
3. train_validation 데이터를 현재는 짝수, 8의배수+1 로 줌. 해당 부분에서 문제가 발생할 수도?
4. 앙상블기법. backbone모델을 resnet18에서 densenet201로 바꾸는 것만으로 정확도가 0.77->0.82로 오름. backbone모델을 다양하게 하여 앙상블하거나 다른 모델을 추가하는 방법?
5. (가장 나중에 하기) 파라미터 튜닝. 성능좋은 모델일수록 파라미터 튜닝 효과가 적기에 가장 나중에..

In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.datasets import make_multilabel_classification

X, y = make_multilabel_classification(n_classes=3, random_state=42)

print(X.shape,y.shape)



(100, 20) (100, 3)


#train/test data 압축해제 / 압축해제후 이름 각각 train/test 으로 바꿔야 함 / *경로 수정 필요*

In [1]:
!unzip "/content/drive/MyDrive/ColabNotebooks/multi-label-classification/clear.zip" -d "/content/clear"

Archive:  /content/drive/MyDrive/ColabNotebooks/multi-label-classification/clear.zip
  inflating: /content/clear/1.png    
  inflating: /content/clear/10.png   
  inflating: /content/clear/100.png  
  inflating: /content/clear/1000.png  
  inflating: /content/clear/1001.png  
  inflating: /content/clear/1002.png  
  inflating: /content/clear/1003.png  
  inflating: /content/clear/1004.png  
  inflating: /content/clear/1005.png  
  inflating: /content/clear/1006.png  
  inflating: /content/clear/1007.png  
  inflating: /content/clear/1008.png  
  inflating: /content/clear/1009.png  
  inflating: /content/clear/101.png  
  inflating: /content/clear/1010.png  
  inflating: /content/clear/1011.png  
  inflating: /content/clear/1012.png  
  inflating: /content/clear/1013.png  
  inflating: /content/clear/1014.png  
  inflating: /content/clear/1015.png  
  inflating: /content/clear/1016.png  
  inflating: /content/clear/1017.png  
  inflating: /content/clear/1018.png  
  inflating: /content/

In [2]:
!unzip "/content/drive/MyDrive/ColabNotebooks/multi-label-classification/test_real.zip" -d "/content"

Archive:  /content/drive/MyDrive/ColabNotebooks/multi-label-classification/test_real.zip
  inflating: /content/test/TEST_00000.jpg  
  inflating: /content/test/TEST_00001.jpg  
  inflating: /content/test/TEST_00002.jpg  
  inflating: /content/test/TEST_00003.jpg  
  inflating: /content/test/TEST_00004.jpg  
  inflating: /content/test/TEST_00005.jpg  
  inflating: /content/test/TEST_00006.jpg  
  inflating: /content/test/TEST_00007.jpg  
  inflating: /content/test/TEST_00008.jpg  
  inflating: /content/test/TEST_00009.jpg  
  inflating: /content/test/TEST_00010.jpg  
  inflating: /content/test/TEST_00011.jpg  
  inflating: /content/test/TEST_00012.jpg  
  inflating: /content/test/TEST_00013.jpg  
  inflating: /content/test/TEST_00014.jpg  
  inflating: /content/test/TEST_00015.jpg  
  inflating: /content/test/TEST_00016.jpg  
  inflating: /content/test/TEST_00017.jpg  
  inflating: /content/test/TEST_00018.jpg  
  inflating: /content/test/TEST_00019.jpg  
  inflating: /content/test/TEST

In [3]:
!unzip "/content/drive/MyDrive/ColabNotebooks/multi-label-classification/train.zip" -d "/content"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/train - ║╣╗ч║╗/TRAIN_27994.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_27995.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_27996.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_27997.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_27998.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_27999.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28000.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28001.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28002.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28003.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28004.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28005.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28006.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28007.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28008.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28009.jpg  
  inflating: /content/train - ║╣╗ч║╗/TRAIN_28010.jpg  
  inflating: /content/train -

In [4]:
!unzip "/content/drive/MyDrive/ColabNotebooks/multi-label-classification/background_0126.zip" -d "/content"

Archive:  /content/drive/MyDrive/ColabNotebooks/multi-label-classification/background_0126.zip
  inflating: /content/background_0126/bg_1.jpg  
  inflating: /content/background_0126/bg_2.jpg  
  inflating: /content/background_0126/bg_3.jpg  
  inflating: /content/background_0126/bg_4.jpg  
  inflating: /content/background_0126/bg_5.jpg  
  inflating: /content/background_0126/bg_6.jpg  
  inflating: /content/background_0126/bg_7.jpg  
  inflating: /content/background_0126/furniture-g8e37ab745_640.jpg  
  inflating: /content/background_0126/interior-g89acd14e9_640.jpg  
  inflating: /content/background_0126/living-room-g02bb7017c_640.jpg  
  inflating: /content/background_0126/office-g828daa8df_640.jpg  
  inflating: /content/background_0126/seminar-gddcfc4db4_640.jpg  
  inflating: /content/background_0126/shop-g92a41d9bd_640.jpg  
  inflating: /content/background_0126/study-g90caf672e_640.jpg  


# 모듈 설치 / import

In [5]:
%cd /content/drive/MyDrive/ColabNotebooks/multi-label-classification

/content/drive/MyDrive/ColabNotebooks/multi-label-classification


In [6]:
!pip install timm pytorch_lightning wandb randaugment positional_encodings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=74ae30120260907221a0477c961f42dfcfc248f59461

In [7]:
import torch
import math
import timm
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
import pytorch_lightning as pl
import wandb

from q2l_labeller.data.cutmix import CutMixCollator
import os
import torch
import pandas as pd
import pytorch_lightning as pl
import torchvision.transforms as transforms
from randaugment import RandAugment
from torch.utils.data import DataLoader

import torch.utils.data as data
from PIL import Image
import numpy as np

from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy, precision
import torchmetrics.functional as tf
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.callbacks import StochasticWeightAveraging
from positional_encodings.torch_encodings import PositionalEncodingPermute2D, Summer

from q2l_labeller.data.coco_data_module import COCODataModule
from q2l_labeller.pl_modules.query2label_train_module import Query2LabelTrainModule
from q2l_labeller.data.cutmix import CutMixCriterion
from q2l_labeller.loss_modules.simple_asymmetric_loss import AsymmetricLoss

In [8]:
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_rw_224',
 'coatnet_1_rw_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_rw_224',
 'coatnet_rmlp_1_rw_224',
 'coatnet_rmlp_2_rw_224',
 'coatnet_rmlp_nano_rw_224',
 'coatnext_nano_rw_224',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_atto',
 'convnext_atto_

# model structure

In [13]:
class TimmBackbone(nn.Module): ##이미 사전학습된 resnet18 모델 가져오기
    def __init__(self, model_name):
        super().__init__()

        # Creating the model in this way produces unpooled, unclassified features
        self.model = timm.create_model(
            model_name, 
            pretrained=True,
            num_classes=0,
            global_pool='')       

    def forward(self, x):
        
        out = self.model(x)
        
        return out

class Query2Label(nn.Module): ##논문 개념만 이해하고 복붙..
    """Modified Query2Label model

    Unlike the model described in the paper (which uses a modified DETR 
    transformer), this version uses a standard, unmodified Pytorch Transformer. 
    Learnable label embeddings are passed to the decoder module as the target 
    sequence (and ultimately is passed as the Query to MHA).
    """
    def __init__(
        self, model, conv_out, num_classes, hidden_dim=256, nheads=8, 
        encoder_layers=6, decoder_layers=6, use_pos_encoding=False):
        """Initializes model

        Args:
            model (str): Timm model descriptor for backbone.
            conv_out (int): Backbone output channels.
            num_classes (int): Number of possible label classes
            hidden_dim (int, optional): Hidden channels from linear projection of
            backbone output. Defaults to 256.
            nheads (int, optional): Number of MHA heads. Defaults to 8.
            encoder_layers (int, optional): Number of encoders. Defaults to 6.
            decoder_layers (int, optional): Number of decoders. Defaults to 6.
            use_pos_encoding (bool, optional): Flag for use of position encoding. 
            Defaults to False.
        """        
        
        super().__init__()

        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        self.use_pos_encoding = use_pos_encoding

        self.backbone = TimmBackbone(model)
        self.conv = nn.Conv2d(conv_out, hidden_dim, 1)
        self.transformer = nn.Transformer(
            hidden_dim, nheads, encoder_layers, decoder_layers)

        if self.use_pos_encoding:
            # returns the encoding object
            self.pos_encoder = PositionalEncodingPermute2D(hidden_dim)

            # returns the summing object
            self.encoding_adder = Summer(self.pos_encoder)

        # prediction head
        self.classifier = nn.Linear(num_classes * hidden_dim, num_classes)

        # learnable label embedding
        self.label_emb = nn.Parameter(torch.rand(1, num_classes, hidden_dim))

    def forward(self, x):
        """Passes batch through network

        Args:
            x (Tensor): Batch of images

        Returns:
            Tensor: Output of classification head
        """        
        # produces output of shape [N x C x H x W]
        out = self.backbone(x)
        
        # reduce number of feature planes for the transformer
        h = self.conv(out)
        B, C, H, W = h.shape

        # add position encodings
        if self.use_pos_encoding:
            
            # input with encoding added
            h = self.encoding_adder(h*0.1)

        # convert h from [N x C x H x W] to [H*W x N x C] (N=batch size)
        # this corresponds to the [SIZE x BATCH_SIZE x EMBED_DIM] dimensions 
        # that the transformer expects
        h = h.flatten(2).permute(2, 0, 1)
        
        # image feature vector "h" is sent in after transformation above; we 
        # also convert label_emb from [1 x TARGET x (hidden)EMBED_SIZE] to 
        # [TARGET x BATCH_SIZE x (hidden)EMBED_SIZE]
        label_emb = self.label_emb.repeat(B, 1, 1)
        label_emb = label_emb.transpose(0, 1)
        h = self.transformer(h, label_emb).transpose(0, 1)
        
        # output from transformer was of dim [TARGET x BATCH_SIZE x EMBED_SIZE];
        # however, we transposed it to [BATCH_SIZE x TARGET x EMBED_SIZE] above.
        # below we reshape to [BATCH_SIZE x TARGET*EMBED_SIZE].
        #
        # next, we project transformer outputs to class labels
        h = torch.reshape(h,(B, self.num_classes * self.hidden_dim))

        return self.classifier(h)

class Query2LabelTrainModule(pl.LightningModule):
    def __init__(
        self,
        data,
        backbone_desc,
        conv_out_dim,
        hidden_dim,
        num_encoders,
        num_decoders,
        num_heads,
        batch_size,
        image_dim,
        learning_rate,
        momentum,
        weight_decay,
        n_classes,
        thresh=0.5,
        use_cutmix=False,
        use_pos_encoding=False,
        loss="BCE",
    ):
        super().__init__()

        # Key parameters
        self.save_hyperparameters(ignore=["model", "data"])
        self.data = data
        self.model = Query2Label(
            model=backbone_desc,
            conv_out=conv_out_dim,
            num_classes=n_classes,
            hidden_dim=hidden_dim,
            nheads=num_heads,
            encoder_layers=num_encoders,
            decoder_layers=num_decoders,
            use_pos_encoding=use_pos_encoding,
        )
        if loss == "BCE":
            self.base_criterion = nn.BCEWithLogitsLoss()
        elif loss == "ASL":
            self.base_criterion = AsymmetricLoss(gamma_neg=1, gamma_pos=0)

        self.criterion = CutMixCriterion(self.base_criterion)

    def forward(self, x):
        x = self.model(x)
        return x

    def evaluate(self, batch, stage=None): ##여러 평가지표, f1score이 가장 중요
        x, y = batch
        y_hat = self(x)
        loss = self.base_criterion(y_hat, y.type(torch.float))

        rmap = tf.retrieval_average_precision(y_hat, y.type(torch.int))

        category_prec = precision(
            y_hat,
            y.type(torch.int),
            average="macro",
            num_classes=self.hparams.n_classes,
            threshold=self.hparams.thresh,
            #multiclass=False,
            task='binary'
        )
        category_recall = tf.recall(
            y_hat,
            y.type(torch.int),
            average="macro",
            num_classes=self.hparams.n_classes,
            threshold=self.hparams.thresh,
            #multiclass=False,
            task='binary'
        )
        category_f1 = tf.f1_score(
            y_hat,
            y.type(torch.int),
            average="macro",
            num_classes=self.hparams.n_classes,
            threshold=self.hparams.thresh,
            #multiclass=False,
            task='binary'
        )

        overall_prec = precision(
            y_hat, y.type(torch.int), threshold=self.hparams.thresh,#multiclass=False,
            task='binary'
        )
        overall_recall = tf.recall(
            y_hat, y.type(torch.int), threshold=self.hparams.thresh, #multiclass=False,
            task='binary'
        )
        overall_f1 = tf.f1_score(
            y_hat, y.type(torch.int), threshold=self.hparams.thresh, #multiclass=False,
            task='binary'
        )

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_rmap", rmap, prog_bar=True, on_step=False, on_epoch=True)

            self.log(f"{stage}_cat_prec", category_prec, prog_bar=True)
            self.log(f"{stage}_cat_recall", category_recall, prog_bar=True)
            self.log(f"{stage}_cat_f1", category_f1, prog_bar=True)

            self.log(f"{stage}_ovr_prec", overall_prec, prog_bar=True)
            self.log(f"{stage}_ovr_recall", overall_recall, prog_bar=True)
            self.log(f"{stage}_ovr_f1", overall_f1, prog_bar=True)

            # log prediction examples to wandb
            """
            pred = self.model(x)
            pred_keys = pred[0].sigmoid().tolist()
            pred_keys = [0 if p < self.hparams.thresh else 1 for p in pred_keys]


            mapper = cc.COCOCategorizer()
            pred_lbl = mapper.get_labels(pred_keys)
            
            try:
                self.logger.experiment.log({"val_pred_examples": [wandb.Image(x[0], caption=pred_lbl)]})
            except AttributeError:
                pass
            """

    def training_step(self, batch, batch_idx):
        if self.hparams.use_cutmix:
            x, y = batch
            y_hat = self(x)
            # y1, y2, lam = y
            loss = self.criterion(y_hat, y)

        else:
            x, y = batch
            y_hat = self(x)
            loss = self.base_criterion(y_hat, y.type(torch.float))
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.hparams.learning_rate,
            betas=(0.9, 0.999),
            weight_decay=self.hparams.weight_decay,
        )

        lr_scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                self.hparams.learning_rate,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=len(self.data.train_dataloader()),
                anneal_strategy="cos",
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        # return optimizer
class COCODataModule(pl.LightningDataModule):
    """Datamodule for Lightning Trainer"""

    def __init__(
        self,
        data_dir,
        img_size,
        batch_size=128,
        num_workers=0,
        use_cutmix=False,
        cutmix_alpha=1.0,
    ) -> None:
        """_summary_

        Args:
            data_dir (str): Location of data.
            img_size (int): Desired size for transformed images.
            batch_size (int, optional): Dataloader batch size. Defaults to 128.
            num_workers (int, optional): Number of CPU threads to use. Defaults to 0.
            use_cutmix (bool, optional): Flag to enable Cutmix augmentation. Defaults to False.
            cutmix_alpha (float, optional): Defaults to 1.0.
        """
        super().__init__()
        self.data_dir = data_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.use_cutmix = use_cutmix
        self.cutmix_alpha = cutmix_alpha
        self.collator = torch.utils.data.dataloader.default_collate
    def prepare_data(self) -> None:
        """Loads metadata file and subsamples it if requested"""
        pass

    def setup(self, stage=None) -> None:
        """Creates train, validation, test datasets

        Args:
            stage (str, optional): Stage. Defaults to None.
        """
        normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
        # mean=[0, 0, 0],
        # std=[1, 1, 1])

        train_transforms = transforms.Compose( ########################################################################################################
            [ #soft_augment
                transforms.Resize((400, 400)),
                transforms.RandomHorizontalFlip(p=0.5), #좌우대칭
                transforms.RandomApply([transforms.ColorJitter(0.02, 0.02, 0.02, 0.02)], p=0.5), # 밝기,대비,채도
                transforms.RandomApply([
                    transforms.RandomResizedCrop(400, scale=(0.8, 1.0),interpolation=transforms.InterpolationMode.BICUBIC)
                ], p=0.75), #일부분 확대 
                
                transforms.ToTensor(),
                normalize,

            ]
        )

        test_transforms = transforms.Compose(
            [
                transforms.Resize((self.img_size, self.img_size)),
                transforms.ToTensor(),
                normalize,
            ]
        )

        self.train_set = mydataset(
            image_dir=self.data_dir,
            input_transform=train_transforms,
            split = 'train'
        )
        self.val_set = mydataset(
            image_dir=self.data_dir,
            input_transform = test_transforms,
            split = 'validation'
        )
        """
        self.test_set = CoCoDataset( 
            image_dir=(self.data_dir+"images/test2014"),
            anno_path=(self.data_dir+"annotations/instances_test2014.json"),
            input_transform=test_transforms,
            labels_path=(self.data_dir+"annotations/labels_test2014.npy"))
        """

        if self.use_cutmix:
            self.collator = CutMixCollator(self.cutmix_alpha)

    def get_num_classes(self):
        """Returns number of classes

        Returns:
            int: number of classes
        """
        return len(self.classes)

    def train_dataloader(self) -> DataLoader:
        """Creates and returns training dataloader

        Returns:
            DataLoader: Training dataloader
        """
        return DataLoader(
            self.train_set,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            collate_fn=self.collator,
        )

    def val_dataloader(self) -> DataLoader:
        """Creates and returns validation dataloader

        Returns:
            DataLoader: Validation dataloader
        """
        return DataLoader(
            self.val_set,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
        )

    def test_dataloader(self) -> DataLoader:
        """Creates and returns test dataloader

        Returns:
            DataLoader: Test dataloader
        """
        return DataLoader(
            self.val_set,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
        )

        """ #custom_augment 정확도90퍼짜리 ---------------------------------------------------->  custom_augment1 <----------------------------
            [
                transforms.Resize((self.img_size, self.img_size)),
                #RandAugment(),
                transforms.RandomHorizontalFlip(p=0.5), #좌우대칭
                transforms.RandomApply( 
                [
                    transforms.RandomAffine( #x,y축으로 사진을 늘림
                        degrees=10,
                        translate=(0.1, 0.1),
                        scale=(0.9, 1.1),
                        shear=10,
                        fill=255,
                        interpolation=transforms.InterpolationMode.BICUBIC,
                    )
                ],
                p=0.7,
                ),
                transforms.RandomApply([transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)], p=0.3), # 밝기,대비,채도
                transforms.RandomApply([transforms.RandomPerspective(distortion_scale=0.2,fill=0)], p=0.2 ), #시점변환
                transforms.RandomApply([
                    transforms.RandomResizedCrop(self.img_size, scale=(0.9, 1.0),interpolation=transforms.InterpolationMode.BICUBIC)
                ], p=0.75), #일부분 확대
                transforms.RandomRotation(3), #기울이기
                 
                transforms.AugMix(), #_--------------------> 빼는게 좋을듯
                
                transforms.ToTensor(),
                normalize,
            ]
        )
       """

# 경로 설정

In [14]:
%cd /content
train_csv = '/content/drive/MyDrive/ColabNotebooks/multi-label-classification/train.csv'
test_csv = '/content/drive/MyDrive/ColabNotebooks/multi-label-classification/test.csv'
best_model_path = '/content/drive/MyDrive/ColabNotebooks/multi-label-classification/training/model_inception_bg_customaug/checkpoints/epoch=23-step=6192.ckpt'

#best_model_path의 경우 ckpt파일 경로를 올려야 함

/content


# edit train,vali,test dataset / change background

In [ ]:
param_dict = {               #파라미터 아직 안건들였음
    "backbone_desc": 'vgg19', #'inception_v3', #'resnet101', #"densenet201",#
    "conv_out_dim":  1920, #2048,  #2048, #1920,
    "hidden_dim": 256, 
    "num_encoders":1,
    "num_decoders":2,
    "num_heads":8,
    "batch_size": 32, #64,
    "image_dim": 224,      #224,
    "learning_rate":0.0001, 
    "momentum":0.9,
    "weight_decay":0.01, 
    "n_classes":10,
    "thresh":0.5,
    "use_cutmix":True,
    "use_pos_encoding":False,
    "loss":"ASL"
}

In [ ]:
import cv2 ###################################해당 파트는 vscode에서 실행해야 함. #################
import numpy as np
from matplotlib import pyplot as plt
import glob
#def onChange(pos):
#    pass

#cv2.namedWindow("test")
#cv2.createTrackbar("threshold", "test", 0, 255, onChange)
#cv2.setTrackbarPos("threshold", "test", 127)



    

def change_background(adress):
    #thresh = cv2.getTrackbarPos("threshold", "test")

    bg_transforms = transforms.Compose([
      transforms.RandomHorizontalFlip(p=0.5), #좌우대칭
      transforms.RandomApply([transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)], p=0.5), # 밝기,대비,채도
      transforms.RandomResizedCrop(400, scale=(0.2, 1.0),interpolation=transforms.InterpolationMode.BICUBIC), #일부분 확대
    ])


    img = cv2.imread(adress)

    img_bin = np.array(img[:, :, 0] < 222, dtype=np.uint8) * 255 ##############여기도 222숫자 수정가능
    img_blur = cv2.medianBlur(img_bin, 5)
    img_filter = img.copy()
    img_filter[img_blur == 0] = 0

    #cv2.imshow("img", img)
    #cv2.imshow("test", img_bin)
    #cv2.imshow("img_blur", img_blur)
    #cv2.imshow("img_filter", img_filter)
    
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img_blur)
    stats = np.array(stats)
    stats[0, -1] = 0
    label_idx = np.argmax(stats[:, -1])
    img_mask = labels == label_idx
    img_final = img_filter.copy()
    img_final[~img_mask] = (0, 0, 0)

    #background_loc = '/content/bg_' + str(num) + '.jpg'
    bg_list=[]
    bg_path = glob.glob('/content/clear/*.png')

    num=np.random.randint(0,len(bg_path))
    
    img_back = cv2.imread(bg_path[num])
    img_back = cv2.resize(img_back , img.shape[::-1][1:])

    cv2.imwrite('/content/y.jpg',img_back)
    y=Image.open('/content/y.jpg').convert("RGB")
    img_back = np.array(bg_transforms(y))
    img_back = cv2.cvtColor(img_back,cv2.COLOR_BGR2RGB)

    img_back[img_mask] = (0, 0, 0)
    img_final += img_back
    #cv2.imshow("img_final", img_final)
    #cv2.waitKey()
    cv2.imwrite('/content/x.jpg',img_final)
    x=Image.open('/content/x.jpg').convert("RGB")
    return x

In [ ]:
class mydataset(data.Dataset): #논문의 cocodataset를 수정한 것. __getitem__과 __len__을 override하는 것이 필수
#index가 주어졌을 때 해당 값의 라벨([0,1,0,0,0,0,1]값)과 rgb image(3x224x224)를 리턴함
    def __init__(self,
                image_dir='',
                input_transform=None,
                used_category=-1,
                split='train'
                ):
      
        self.df = pd.read_csv(train_csv)

        if split=='train': #train, validation 80 : 20 비율 / 이 부분은 나중에 개선 가능
          self.len = int(len(self.df)) #홀수만 설정
          self.start = 0
          self.step=2
        if split=='validation':
          self.len = int(len(self.df)) #validation 너무 많으면 오래걸리니까 8의 배수로
          self.start = 1
          self.step=8

        self.len = int(self.len)

        self.input_transform = input_transform
        self.used_category = used_category
        
        self.labels = []
        self.image = []

        #%cd /content ####################### 세션으로 이동해주어야 아래 cur_img_path가 원활히 작동 ############
        
        for i in range( self.start , self.len , self.step ):
            cur_id = self.df['id'][i]
            cur_img_path = self.df['img_path'][i]
            cur_value = np.array(self.df.iloc[i,2:]) #1d array (10,1)
            
            cur_value = (cur_value > 0).astype(np.float64) #change int to float
            self.labels.append(cur_value)
            
            #cur_img = Image.open(cur_img_path).convert("RGB")
            cur_img = change_background(cur_img_path)

            self.image.append(cur_img)
            
            if( int(i/10)%250 == 0 ):
                print( f" {int(i*100/self.len)}% ", end='')
        print()

    def __getitem__(self, index): #has to override
        input = self.image[index]
        if self.input_transform:
            input = self.input_transform(input) # reshape image
            
        return input, self.labels[index]
    
    def __len__(self): #has to override
        return int( (self.len-self.start)/self.step) #데이터 길이

# 실행

In [ ]:
if __name__ == "__main__": # 파일 다운로드 약 15분 / 학습 약 50분 
    pl.seed_everything(42)
    img_data_dir = ''

    coco = COCODataModule(
    img_data_dir,
    img_size=param_dict["image_dim"],
    batch_size=param_dict["batch_size"],
    num_workers=4,
    use_cutmix=param_dict["use_cutmix"],
    cutmix_alpha=1.0)
    
    param_dict["data"] = coco
    
    pl_model = Query2LabelTrainModule(**param_dict)

    #pl_model = pl_model.load_from_checkpoint(best_model_path,**param_dict) ##############
    
    trainer = pl.Trainer(
        max_epochs=24,
        precision=16,
        accelerator='auto', 
        devices=1,
        default_root_dir="training/checkpoints/",
        callbacks=[TQDMProgressBar(refresh_rate=10)])
    trainer.fit(pl_model, param_dict["data"])

INFO:lightning_fabric.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


 0%  0%  0%  0%  0%  7%  7%  7%  7%  7%  15%  15%  15%  15%  15%  22%  22%  22%  22%  22%  30%  30%  30%  30%  30%  37%  37%  37%  37%  37%  45%  45%  45%  45%  45%  53%  53%  53%  53%  53%  60%  60%  60%  60%  60%  68%  68%  68%  68%  68%  75%  75%  75%  75%  75%  83%  83%  83%  83%  83%  90%  90%  90%  90%  90%  98%  98%  98%  98%  98% 
 0%  0%  7%  15%  15%  22%  30%  30%  37%  45%  45%  53%  60%  60%  68%  75%  75%  83%  90%  90%  98% 


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type           | Params
--------------------------------------------------
0 | model          | Query2Label    | 23.1 M
1 | base_criterion | AsymmetricLoss | 0     
--------------------------------------------------
23.1 M    Trainable params
0         Non-trainable params
23.1 M    Total params
46.173    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


# 무작위 인덱스 결과 확인

In [ ]:
#test best model


def test_random_index(model,trainer):
      
        normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )

        input_transform = transforms.Compose(
            [
                transforms.Resize((param_dict['image_dim'],param_dict['image_dim'])),#img size
                transforms.ToTensor(),
                normalize,
            ]
        )
        
        df = pd.read_csv(train_csv)
        _len = int(len(df))
        
        labels = []
        image = []
        for x in range(5):
            i = np.random.randint(0,_len) #무작위 index 0~34999 골라서 정답과 예측값을 알려줌
            cur_id = df['id'][i]
            
            cur_img_path = df['img_path'][i]
            cur_img = Image.open(cur_img_path).convert("RGB")
            input = cur_img
            input = input_transform(input) # reshape image
            
            dataload = DataLoader(
              [input],
              batch_size=1,
              shuffle=False,
            )
            
            ans = trainer.predict(model, dataload )
            
            pred_keys = ans[0][0].tolist()
            pred_keys = [0 if p < 0.5 else 1 for p in pred_keys]


            cur_value = np.array(df.iloc[i,2:]) #1d array (10,1)
            cur_value = (cur_value > 0).astype(np.float64) #change int to float
            
            k=['A','B','C','D','E','F','G','H','I','J','K']

            print(cur_id)
            for j in range(1,11):
              print(k[j],end='    ')
            print()
            for j in range(10):
              pred=cur_value[j]
              print(pred*1.0,end='  ')
            print()
            for j in range(10):
              pred=pred_keys[j]
              print(pred*1.0,end='  ')
            print()
            for j in ans[0][0].tolist():
              if(j>=0):
                print(int(j*10)/10,end='  ')
              else:
                print(int(j*10)/10,end=' ')
            print()
            #print(f"model's predict_raw:{ans[0][0].tolist()}")
            #print(f"model's prediction: {pred_keys}")
            #print(f"real value        : {cur_value}")

if True:
    pl.seed_everything(15)
    img_data_dir = ''
    coco = COCODataModule(
    img_data_dir,
    img_size=param_dict["image_dim"],
    batch_size=param_dict["batch_size"],
    num_workers=2,
    use_cutmix=param_dict["use_cutmix"],
    cutmix_alpha=1.0)
    
    param_dict["data"] = coco
    
    pl_model = Query2LabelTrainModule(**param_dict)

    pl_model = pl_model.load_from_checkpoint(best_model_path,**param_dict)

    trainer = pl.Trainer(
        accelerator='auto', ## 
        devices=1,
        )
    
    test_random_index(pl_model,trainer)

INFO:lightning_fabric.utilities.seed:Global seed set to 15
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


TRAIN_07624
B    C    D    E    F    G    H    I    J    K    
0.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  0.0  0.0  
0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  
-0.3 -0.5 -0.4 1.7  2.3  0.1  -0.3 1.7  -0.1 -0.1 


Predicting: 0it [00:00, ?it/s]

...

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


TRAIN_30411
B    C    D    E    F    G    H    I    J    K    
0.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  
0.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  
-1.1 -1.3 3.3  -1.3 2.5  2.9  2.2  -0.8 -0.4 -0.4 


Predicting: 0it [00:00, ?it/s]

TRAIN_06229
B    C    D    E    F    G    H    I    J    K    
0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  
0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  
0.0 1.2  0.3  2.1  2.2  2.2  -1.0 -0.7 -1.0 -0.7 


#test파일 데이터로 실험

In [ ]:
from google.colab import output


real_ans_demo = [
    [2,7,9],
    [2,6],
    [1,2,4,5,6,8,10],
    [5,6,8,9],
    [2,5],
    [1,2,3,5,8,9],
    [10,9,8,6,3,4],
    [2,3,4,6,7,10],
    [1,2,3,4,5,6,10],
    [1,2,7]
]
_len=10

real_ans = np.zeros( [100,10] , dtype=int)

for i in range(_len):
  for j in real_ans_demo[i]:
    real_ans[i][j-1]=1

def create_test(model,trainer,_len): #약 20분 걸림
      
        normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )

        input_transform = transforms.Compose(
            [
                transforms.Resize((param_dict['image_dim'],param_dict['image_dim'])),#img size
                transforms.ToTensor(),
                normalize,
            ]
        )
        
        df = pd.read_csv(test_csv) ############ test.csv 경로 설정 ############
        
        cnt=0

        for i in range(_len): #출력창 지우는거 필요할 듯, 넘 길어진다
            cur_id = df['id'][i]

            cur_img_path = df['img_path'][i]
            cur_img = Image.open(cur_img_path).convert("RGB")
            input = cur_img
            input = input_transform(input) # reshape image
            
            dataload = DataLoader(
              [input],
              batch_size=1,
              shuffle=False,
            )
            
            ans = trainer.predict(model, dataload )
            
            pred_keys = ans[0][0].tolist()
            pred_keys = [0 if p < 0.5 else 1 for p in pred_keys]

            k=['A','B','C','D','E','F','G','H','I','J','K']

            print(cur_id)
            for j in range(1,11):
              print(k[j],end='    ')
            print()
            for j in range(10):
              pred=real_ans[i][j]
              print(pred*1.0,end='  ')
            print()
            for j in range(10):
              pred=pred_keys[j]
              print(pred*1.0,end='  ')
              if(pred!=real_ans[i][j]):
                cnt+=1
            print()
            for j in ans[0][0].tolist():
              if(j>=0):
                print(int(j*10)/10,end='  ')
              else:
                print(int(j*10)/10,end=' ')
            print()
        
        print(cnt)

if True:
    pl.seed_everything(42)
    img_data_dir = ''
    coco = COCODataModule(
    img_data_dir,
    img_size=param_dict["image_dim"],
    batch_size=param_dict["batch_size"],
    num_workers=2,
    use_cutmix=param_dict["use_cutmix"],
    cutmix_alpha=1.0)
    
    param_dict["data"] = coco
    
    pl_model = Query2LabelTrainModule(**param_dict)

    pl_model = pl_model.load_from_checkpoint(best_model_path,**param_dict)
    ##################################### best_model 경로 ################
    trainer = pl.Trainer(
        accelerator='auto', ## 
        devices=1,
        )
    create_test( pl_model , trainer,_len)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


TEST_00000
B    C    D    E    F    G    H    I    J    K    
0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  
0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  
-0.7 2.7  -1.1 -1.5 -1.2 -1.2 3.6  -1.3 3.7  -1.4 


Predicting: 0it [00:00, ?it/s]

...

Predicting: 0it [00:00, ?it/s]

TEST_00008
B    C    D    E    F    G    H    I    J    K    
1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  
0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  
-0.6 1.3  1.6  2.3  1.4  0.9  0.7  -0.8 -0.4 1.4  


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

TEST_00009
B    C    D    E    F    G    H    I    J    K    
1.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  
1.7  -1.3 -0.8 -1.4 2.4  -1.6 2.3  -0.5 -0.6 -0.7 
16


#실제 test파일 생성 / 세션에 생성됨

In [ ]:
from google.colab import output
def create_test(model,trainer): #약 20분 걸림
      
        normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )

        input_transform = transforms.Compose(
            [
                transforms.Resize((param_dict['image_dim'],param_dict['image_dim'])),#img size
                transforms.ToTensor(),
                normalize,
            ]
        )
        
        df = pd.read_csv(test_csv) ############ test.csv 경로 설정 ############
        _len = int(len(df))
        
        id = []
        label = []
        for i in range(10):
          label.append([])
        for i in range(_len): #출력창 지우는거 필요할 듯, 넘 길어진다
            cur_id = df['id'][i]

            cur_img_path = df['img_path'][i]
            cur_img = Image.open(cur_img_path).convert("RGB")
            input = cur_img
            input = input_transform(input) # reshape image
            
            dataload = DataLoader(
              [input],
              batch_size=1,
              shuffle=False,
            )
            
            ans = trainer.predict(model, dataload )
            
            pred_keys = ans[0][0].tolist()
            pred_keys = [0 if p < 0.5 else 1 for p in pred_keys] ############

            id.append(cur_id)
            for j in range(10):
              label[j].append(pred_keys[j])    
            
            if(i%20==0):
              output.clear()
              print(f"{i/_len}%...        ")

        return id,label

if True:
    pl.seed_everything(24)
    img_data_dir = ''
    coco = COCODataModule(
    img_data_dir,
    img_size=param_dict["image_dim"],
    batch_size=param_dict["batch_size"],
    num_workers=2,
    use_cutmix=param_dict["use_cutmix"],
    cutmix_alpha=1.0)
    
    param_dict["data"] = coco
    
    pl_model = Query2LabelTrainModule(**param_dict)

    pl_model = pl_model.load_from_checkpoint(best_model_path,**param_dict)
    ##################################### best_model 경로 ################
    trainer = pl.Trainer(
        accelerator='auto', ## 
        devices=1,
        )
    id,label=create_test( pl_model , trainer)

In [ ]:
data = {
    'id' : id,
    'A' : label[0],
    'B' : label[1],
    'C' : label[2],
    'D' : label[3],
    'E' : label[4],
    'F' : label[5],
    'G' : label[6],
    'H' : label[7],
    'I' : label[8],
    'J' : label[9]
}

fin = pd.DataFrame(data)

fin.index = fin['id']
fin = fin[['A','B','C','D','E','F','G','H','I','J']]

fin.to_csv('submit_inception_0127_upgrade_background_custom_augment.csv')

#ensemble_demo

In [ ]:
%cd
y = pd.read_csv('/content/submit_ensemble_demo_0127.csv')
x = pd.read_csv('/content/densenet201_demo.csv')
#z = pd.read_csv('/content/submit_inception_0127_upgrade_background_custom_augment.csv')


labels = ['A','B','C','D','E','F','G','H','I','J']
cnt=0
temp=0
for i in range(1460):
  for j in range(10):
    temp =  y[labels[j]][i] + x[labels[j]][i] #z[ labels[j] ][i] +
    if(temp>1):
      x[labels[j]][i]=1
      cnt+=1
    else:
      #if(x[labels[j]][i]==1):
      #  cnt+=1
      x[labels[j]][i]=0
print(cnt)
x.to_csv('submit_ensemble_demo.csv')

/root
1532


In [ ]:
#있는데 없다고한거 11개 없는데 있다고한거 4개

real_ans_demo = [
    [2,7,9],
    [2,6],
    [1,2,4,5,6,8,10],
    [5,6,8,9],
    [2,5],
    [1,2,3,5,8,9],
    [10,9,8,6,3,4],
    [2,3,4,6,7,10],
    [1,2,3,4,5,6,10],
    [1,2,7]
]

real_ans = np.zeros( [100,10] , dtype=int)

for i in range(10):
  for j in real_ans_demo[i]:
    real_ans[i][j-1]=1

cnt=0

for i in range(10): #1460
  print(i)
  print('예측    : ',end='')
  for j in range(10):
    cur = labels[j]
    print(x[cur][i],end=' ')
  print()
  print('정답    : ',end='')
  for j in range(10):
    cur = labels[j]
    print(real_ans[i][j],end=' ')
    if(x[cur][i]!=real_ans[i][j]):
      cnt+=1
  print()


print(cnt)

0
예측    : 0 1 0 0 0 0 1 0 1 0 
정답    : 0 1 0 0 0 0 1 0 1 0 
1
예측    : 0 1 0 0 0 1 0 0 0 0 
정답    : 0 1 0 0 0 1 0 0 0 0 
2
예측    : 0 1 0 0 1 1 1 1 0 0 
정답    : 1 1 0 1 1 1 0 1 0 1 
3
예측    : 0 0 0 0 1 1 0 1 1 0 
정답    : 0 0 0 0 1 1 0 1 1 0 
4
예측    : 0 1 0 0 1 0 0 0 0 0 
정답    : 0 1 0 0 1 0 0 0 0 0 
5
예측    : 1 0 0 1 1 0 0 1 1 0 
정답    : 1 1 1 0 1 0 0 1 1 0 
6
예측    : 0 0 1 1 0 1 0 1 1 0 
정답    : 0 0 1 1 0 1 0 1 1 1 
7
예측    : 0 0 1 1 0 1 1 0 0 1 
정답    : 0 1 1 1 0 1 1 0 0 1 
8
예측    : 0 0 1 1 1 0 0 0 0 1 
정답    : 1 1 1 1 1 1 0 0 0 1 
9
예측    : 1 0 0 0 1 0 1 0 0 0 
정답    : 1 1 0 0 0 0 1 0 0 0 
14
